**Project**

MACD Parameter tuning

## Import Libraries

In [44]:
import pandas as pd
import numpy as np
import yfinance as yf
import talib as ta

## Import Dataset

In [45]:
# define asset and date
ticker_name = 'BTC-USD'
start='2021-01-01' 
end='2022-12-31'

In [46]:
raw_df = yf.download(tickers=ticker_name, start=start, end=end)

[*********************100%***********************]  1 of 1 completed


## Optimization

In [105]:
df =raw_df.copy()

In [137]:
def macd_optimization(df, init_number=12, end_number=26, signal=9):
    all_trades_summary = pd.DataFrame(columns=['Average', 'Min', 'Max', 'i', 'j'])
    
    for i in range(init_number, end_number):
        for j in range(init_number+1, end_number+1):
            temp_df = df.copy()
            temp_df['macd'], temp_df['macdsignal'], temp_df['macdhist'] = ta.MACD(temp_df['Close'], fastperiod=i, slowperiod=j, signalperiod=signal)
            # entry signal sma12 > sma26 or macd > 0
            temp_df['signal'] = temp_df['macd'] > 0
            temp_df.ta.tsignals(temp_df.signal, asbool=True, append=True)

            # entry price signal
            temp_df['action_price'] = temp_df.Open.shift(-1)
            
            # result all transaction on i, j 
            trades_table = temp_df.iloc[:,-5:][temp_df.TS_Trades != 0] # Entry & Exit
            trades_table['return'] = trades_table['action_price'].pct_change() # pct_change() of Entry and Exit
            trades_summary = trades_table.loc[trades_table.TS_Exits == True] # real return each trade

            # create temp table for keep return on i, j and then insert into 'all_trades_summary' table
            a = trades_summary['return']
            temp_a = pd.DataFrame(data=[[a.mean(), a.min(), a.max(),i, j]], columns=['Average', 'Min', 'Max', 'i', 'j'])
            
            all_trades_summary = pd.concat([all_trades_summary, temp_a])
            all_trades_summary.dropna(inplace=True) # drop row is NaN (if i and j are same the output is NaN)

    return all_trades_summary

In [143]:
output_df = macd_optimization(df)
output_df.head(50)

,Average,Min,Max,i,j
0,0.001101,-0.209488,0.455416,12,13
0,-0.001022,-0.209488,0.506897,12,14
0,0.011529,-0.209488,0.506897,12,15
0,0.016305,-0.209488,0.506897,12,16
0,0.005943,-0.209488,0.437144,12,17
0,0.000678,-0.209488,0.437144,12,18
0,-0.002930,-0.209488,0.437144,12,19
0,-0.002815,-0.209488,0.437144,12,20
0,-0.005134,-0.209488,0.437144,12,21
0,-0.000503,-0.209488,0.458295,12,22
